In [ ]:
import json
import sqlite3

In [ ]:
db_file = 'C:/Users/david/Documents/Big Data/Yelp/yelp.sqlite'
file_dir = '/Users/david/Documents/Big Data/Yelp/yelp_academic_dataset_user.json'
insert_script_template = 'INSERT INTO {} ({}) VALUES (?{});'  # template for insert command
table_scripts = {}  # dictionary of create table scripts for each table
ignored_keys = {}   # dictionary of keys ignored by each table
string_values = {}  # dictionary of values that need to be converted to strings to input into table

In [ ]:
# connect to database
connection = sqlite3.connect(db_file)
cursor = connection.cursor()

In [ ]:
with open(file_dr) as file
    for i in range(5):
        print file.readline()

# Creating users table

In [ ]:
# user table constants
table_scripts['users'] = '''CREATE TABLE users               
                           (user_id            varchar(22) NOT NULL PRIMARY KEY,  
                            name               varchar(32) NOT NULL,
                            review_count       int(10)     NOT NULL, 
                            yelping_since      date        NOT NULL, 
                            useful             int(10)     NOT NULL, 
                            funny              int(10)     NOT NULL, 
                            cool               int(10)     NOT NULL, 
                            fans               int(10)     NOT NULL, 
                            average_stars      FLOAT(3,2)  NOT NULL, 
                            compliment_hot     int(5)      NOT NULL, 
                            compliment_more    int(5)      NOT NULL, 
                            compliment_profile int(5)      NOT NULL, 
                            compliment_cute    int(5)      NOT NULL, 
                            compliment_list    int(5)      NOT NULL, 
                            compliment_note    int(5)      NOT NULL, 
                            compliment_plain   int(5)      NOT NULL, 
                            compliment_cool    int(5)      NOT NULL, 
                            compliment_funny   int(5)      NOT NULL, 
                            compliment_writer  int(5)      NOT NULL, 
                            compliment_photos  int(5)      NOT NULL, 
                            type               varchar(20) NOT NULL )'''
ignored_keys['users'] = ['friends', 'elite']
string_values = ['user_id', 'name', 'type', 'yelping_since']

In [ ]:
# makes tables
try:
    cursor.execute(table_scripts['users'])
except sqlite3.Error as error:
    print('An error occurred when making table -', error)

In [ ]:
#inserts rows
with open(file_dir, 'r', encoding="utf8") as file:  # encoding specified since I was running into errors when I didn't
    columns = [key for key in json.loads(file.readline()) if key not in ignored_keys['users']]  # used column names
    file.seek(0)  # returns to beginning of file
    for line in file:
        user = json.loads(line)
        data = []
        for key in user:
            if key not in ignored_keys['users']:
                if key in string_values:
                    data.append(str(user[key]))  
                else:                               
                    data.append(user[key])
        cursor.execute(insert_script_template.format('users', ','.join(columns), ', ?' * (len(columns)-1)), data)
    connection.commit()

In [ ]:
# to check if users loaded properly
cursor.execute('SELECT * FROM {} LIMIT 10'.format('users'))
for user in cursor.fetchall():
    print(user, '\n')

# Creating friends table - works untill here

In [ ]:
# creates set friend pairs which consists of sets of friend pairs so that duplicates don't occure
# also takes too long hence the if False to make code skip
if False:
    friend_pairs = set([])
    with open(file_dir, encoding='utf8') as file:
        for line in file:
            user = json.loads(line)
            for friend in user['friends']:
                friend_pairs.add(frozenset((user['user_id'], friend)))

In [ ]:
def binary_search(lst, element):
    """searches for the element in a sorted list"""
    length = len(lst)
    if length is 0:
        return False
    else:
        half = length//2
        if lst[half]==element:
            return True
        elif element<lst[half]:
            return binary_search(lst[:half], element)
        else:
            return binary_search(lst[half + 1:], element)

In [ ]:
def insert_friend(user_id, friend_id):
    cursor.execute(insert_script_template.format('friends', 'user_id, friend_id', ', ?'),
                                  [user_id, friend_id])

In [ ]:
# get list of all user_ids in users
cursor.execute("SELECT user_id FROM {}".format('users'))
user_ids = sorted([user_id[0].replace("-", "") for user_id in cursor.fetchall()])

In [ ]:
#can't get foreign key syntax to work - FORIEGN KEY (user_id) REFERENCES users(user_id)
try: cursor.execute('''CREATE TABLE friends
            ( user_id char(22) NOT NULL,
            friend_id char(22) NOT NULL,
            FORIEGN KEY (user_id) REFERENCES users(user_id),
            PRIMARY KEY (user_id, friend_id))''')
except sqlite3.Error as error:
    print('An error occurred when making table -', error)

In [ ]:
with open(file_dir, encoding='utf8') as file:
    for line in file:
        user = json.loads(line)
        for friend in user['friends']:
            if binary_search(user_ids, friend):  # avoids duplicate listings which can only occure if the friend is in
                if friend>user['user_id']:       # the user list by forcing an arbitrary ordering in that case
                    insert_friend(user['user_id'], friend)
                    print('inserting ', friend)
            else:
                insert_friend(user['user_id'], friend)
                print('inserting ', friend)
            
    connection.commit()

In [ ]:
connection.commit()

# Creating elite table

In [ ]:
#list of all years that a user can be an elit during
with open(file_dir, enconding='utf8') as file:
    years_lst = []
    for line in file:
        user = json.loads(line)
        for year in user['elite']:
            if year not in years_lst:
                years_lst.append(year)

In [ ]:
try: cursor.execute('''CREATE TABLE years_elite
                        (user_id varchar(22) NOT NULL,
                        {})'''.format(', '.join('{} int DEFAULT 0'.format(year) for year in years_lst)))
except sqlite3.Error as error:
    print('An error occurred when making table -', error)

In [ ]:
with open(file_dir, encoding='utf8') as file:
    for line in file:
        elite_years = json.loads(line)['elite']
        cursor.execute(insert_script_template.format('friends', ', '.join(elite_years), ', ?' * (len(elite_years) - 1)),
                        [1]*len(elite_years))
    cursor.commit()

In [ ]:
conection.close()